### HTML 데이터 파싱을 이용한 크롤링
- 네이버 실시간 검색어 순위 
    - HTML 포멧의 문자열을 가져오는 URL 찾음
    - 요청 -> 응답(HTML 포멧(str)의 데이터를 저장)
    - 데이터 파싱
        - bs4 패키지의 BeautifulSoup 클래스를 이용해서 dom 객체를 생성
        - dom 객체에서 우리가 원하는 데이터를 css-selector를 이용해서 수집
        - 데이터 프레임으로 만들기

In [1]:
import requests
from bs4 import BeautifulSoup

In [6]:
# 1. requests 이용하여 HTML 코드 문자열 데이터 가져오기
response = requests.get("https://www.naver.com/")
response

<Response [200]>

In [8]:
# 2. BeautifulSoup을 이용하여 css selector를 사용할수 있는 객체로 파싱
dom = BeautifulSoup(response.content, "html.parser")

In [13]:
# 3. css selector를 이용하여 원하는 데이터 수집
# select : 여러개의 엘리먼트 객체들을 리스트로 가져옴
# select_one : 하나의 엘리먼트 객체를 가져옴
elements = dom.select(".ah_roll_area > .ah_l > .ah_item")
len(elements)

20

In [16]:
type(elements[0]), elements[0]

(bs4.element.Tag, <li class="ah_item">
 <a class="ah_a" data-clk="lve.keyword" href="#">
 <span class="ah_r">1</span>
 <span class="ah_k">스텝업 입시부적</span>
 </a>
 </li>)

In [18]:
elements[0].select_one(".ah_k").text

'스텝업 입시부적'

In [19]:
# 4. 수집한 데이터를 데이터 프레임으로 만들기

In [21]:
datas = []

for element in elements:
    datas.append({
        "rank": element.select_one(".ah_r").text,
        "keyword": element.select_one(".ah_k").text,
    })
    
naver_keywords = pd.DataFrame(datas)
naver_keywords.tail(3)

,keyword,rank
17,김경수,18
18,수능 국어,19
19,목요일프렌즈타임,20


In [22]:
def naver_keywords():
    
    response = requests.get("https://www.naver.com/")
    dom = BeautifulSoup(response.content, "html.parser")
    elements = dom.select(".ah_roll_area > .ah_l > .ah_item")
    
    datas = []
    for element in elements:
        datas.append({
            "rank": element.select_one(".ah_r").text,
            "keyword": element.select_one(".ah_k").text,
        })
    
    return pd.DataFrame(datas)

#### Quiz
- 다음 실시간 검색어 순위 데이터 수집
- https://daum.net
- 네이버 실시간 검색어와 다음 실시간 검색어 중복 키워드 출력

In [28]:
response = requests.get("https://daum.net")
response

<Response [200]>

In [30]:
dom = BeautifulSoup(response.content, "html.parser")

In [33]:
elements = dom.select(".realtime_part > .list_hotissue.issue_row > li")
len(elements)

10

In [35]:
rank = elements[0].select_one(".ir_wa").text
keyword = elements[0].select_one(".link_issue").text
print(rank, keyword)

1위 수능 시간표


In [36]:
datas = []

for element in elements:
    datas.append({
        "rank": element.select_one(".ir_wa").text,
        "keyword": element.select_one(".link_issue").text,
    })
    
daum_keywords = pd.DataFrame(datas)
daum_keywords.tail(2)

,keyword,rank
8,수능 수학,9위
9,수능 국어,10위


In [38]:
naver_df = naver_keywords()
daum_df = pd.DataFrame(datas)

In [42]:
set(naver_df["keyword"]) & set(daum_df["keyword"])

{'구본영', '김경수', '수능 국어', '수능 수학', '수능 시간표', '수능 영어', '유리 오빠', '이문정'}